## INIT

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import json
from PIL import Image
import os
# print(len(os.listdir('LaTex_data/split_1')))
# Paths
image_folder_path = "LaTex_data/generated_png_images"
mapping_path = "image_formula_mapping.json"
label_to_index_path = "LaTex_data/230k.json"

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
folders = ["split_1", "split_2", "split_3", "split_4", "split_5", 
           "split_6", "split_7", "split_8", "split_9", "split_10"]
  
with open(mapping_path, 'r') as f:
  image_formula_mapping = json.load(f)
keys = list(image_formula_mapping.keys())

# print(image_formula_mapping['0002475406d9932.png'])


## Load Data


In [2]:
import random
class LaTeXDataset(Dataset):
    def __init__(self, image_folder, mapping_file, label_to_index_file, transform=None, max_images=None):
        self.image_folder = image_folder
        self.transform = transform

        # Load mappings and label-to-index dictionary
        with open(mapping_file, 'r') as f:
            self.image_formula_mapping = json.load(f)
        with open(label_to_index_file, 'r') as f:
            self.label_to_index = json.load(f)


        # Apply the image count limit if specified
        # if max_images:
        #     self.image_formula_mapping = dict(list(self.image_formula_mapping.items())[:max_images])

        self.index_to_label = {v: k for k, v in self.label_to_index.items()}
        self.vocab_size = len(self.label_to_index)
        self.formulas = list(self.image_formula_mapping.values())
        self.image_files = [f for f in os.listdir(image_folder) ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
    
        image_name = self.image_files[idx]
        formula = self.image_formula_mapping[str(image_name)]
  
        # Load image
        image_path = os.path.join(self.image_folder, image_name)
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Tokenize the formula into indices
        formula_tokens = formula.split()  # Splitting the formula string by whitespace
        formula_indices = []
        for token in formula_tokens:
            # Map each token to its index; if not found, use a default index (e.g., 0)
            index = self.label_to_index.get(token, 0)  # Assuming 0 is for unknown tokens
            formula_indices.append(int(index))
        
        # Convert the list of indices to a 1D tensor
        return image, torch.tensor(formula_indices, dtype=torch.long)



## Encoder / Decoder

In [3]:


class EncoderCNN(nn.Module):
    def __init__(self, feature_dim):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet50(pretrained=True)  # Example using ResNet50
        for param in resnet.parameters():
            param.requires_grad = False  # Freeze ResNet layers

        # Replace the final fully connected layer with a custom one
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, feature_dim)

    def forward(self, images):
        features = self.resnet(images)  # Shape: [batch_size, feature_dim, 1, 1]
        features = features.view(features.size(0), -1)  # Flatten to [batch_size, feature_dim]
        features = self.fc(features)
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, features, formulas):
        # Embed the input formula tokens
        embeddings = self.embedding(formulas)

        # Concatenate features and embeddings along the sequence dimension
        embeddings = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        
        # Pass through GRU and then through the final linear layer
        gru_out, _ = self.gru(embeddings)
        outputs = self.fc(gru_out)
        return outputs

class ImageToLaTeXModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageToLaTeXModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, formulas):
        # Encode the images
        features = self.encoder(images)  # Shape: [batch_size, feature_dim]
        
        # Decode to generate the LaTeX expression
        outputs = self.decoder(features, formulas[:, :-1])  # Skip the end token
        return outputs


In [4]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Separate images and formulas from the batch
    images, formulas = zip(*batch)

    # Stack images (assumes images are already the same size after transforms)
    images = torch.stack(images)

    # Pad formulas to the length of the longest formula in the batch
    formulas = pad_sequence(formulas, batch_first=True, padding_value=1)  

    return images, formulas


## Save / Load model

In [5]:
import torch

# Assuming 'model' is your model and 'optimizer' is your optimizer
def save_model(model, optimizer, epoch, loss, filename='model.pth'):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, filename)
def load_model(model, optimizer, filename='model.pth'):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return epoch, loss



In [6]:
# Hyperparameters
embed_size = 128
hidden_size = 512
num_epochs = 10
learning_rate = 0.001
batch_size = 32

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((50, 400)),
    transforms.ToTensor()
])

# Load dataset and dataloader


dataset = LaTeXDataset("LaTex_data/split_1" , mapping_path, label_to_index_path, transform)

# Model, loss, and optimizer
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, dataset.vocab_size)
model = ImageToLaTeXModel(encoder, decoder)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 0 is assumed as <PAD> token
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
start_epoch = 0

# Define the paths of your image folders
folders = ["split_1", "split_2", "split_3", "split_4", "split_5", 
           "split_6", "split_7", "split_8", "split_9", "split_10"]

# Function to save model state
def save_training_state(model, optimizer, epoch, folder_idx, loss):
    state = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'folder_idx': folder_idx,
        'loss': loss
    }
    torch.save(state, 'model_checkpoint.pth')

# Function to load model state
def load_training_state(model, optimizer):
    checkpoint = torch.load('model_checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch'], checkpoint['folder_idx'], checkpoint['loss']

# Try to resume from a checkpoint
try:
    start_epoch, start_folder_idx, last_loss = load_training_state(model, optimizer)
    print(f"Resuming training from folder {start_folder_idx+1}, epoch {start_epoch}, with loss {last_loss:.4f}")
except FileNotFoundError:
    print("No saved model found, starting fresh.")
    start_epoch = 0
    start_folder_idx = 0

# Training loop
for folder_idx in range(start_folder_idx, len(folders)):
    print(f"Training on folder: {folders[folder_idx]}")
    dataset = LaTeXDataset("LaTex_data/" + folders[folder_idx], mapping_path, label_to_index_path, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    for epoch in range(start_epoch, num_epochs):
        for i, (images, formulas) in enumerate(dataloader):
            images, formulas = images.to(device), formulas.to(device)
            targets = formulas[:, 1:]

            outputs = model(images, formulas[:, :-1])
            loss = criterion(outputs.view(-1, dataset.vocab_size), targets.contiguous().view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                print(f"Folder [{folder_idx+1}/{len(folders)}], Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(dataloader)}], Loss: {loss.item():.4f}")

            # Save model periodically and at the end of each folder
            if i % 200 == 0 or (i == len(dataloader) - 1):
                save_training_state(model, optimizer, epoch, folder_idx, loss.item())
        
        # Reset start_epoch for next folder
        start_epoch = 0


c:\Users\bibby\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bibby\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\bibby\AppData\Local\Temp\ipykernel_25984\3498543355.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpick

Resuming training from folder 7, epoch 6, with loss 0.7264
Training on folder: split_7


KeyboardInterrupt: 

In [7]:
def decode_formula(indices, index_to_label):
    return ' '.join([index_to_label[str(i.item())] for i in indices if i.item() and str(i.item()) != '1'])  # Skip padding


def validate_model(model, dataloader, criterion, device, index_to_label):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation
        for images, formulas in dataloader:
            images, formulas = images.to(device), formulas.to(device)
            outputs = model(images, formulas[:, :-1])  # Pass images and input sequence

            # Calculate loss
            loss = criterion(outputs.view(-1, outputs.size(-1)), formulas[:, 1:].contiguous().view(-1))
            total_loss += loss.item()

            # Calculate accuracy (if applicable)
            predicted_indices = torch.argmax(outputs, dim=2)  # Get the index of the max log-probability
            correct_predictions += (predicted_indices == formulas[:, 1:].contiguous()).sum().item()
            total_samples += formulas[:, 1:].numel()  # Total number of tokens in the validation batch

            # Print images and predictions
            for i in range(len(images)):
                # Decode the actual and predicted formulas
                actual_formula = decode_formula(formulas[i, 1:], index_to_label)  # Skip <S> token
                predicted_formula = decode_formula(predicted_indices[i, 1:], index_to_label)  # Skip <S> token
                # print(f'Image: {images[i]}')  # This will print the tensor, consider using visualization instead
                print(f'Actual Formula: {actual_formula}')
                print(f'Predicted Formula: {predicted_formula}')
                print('-' * 50)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0.0

    return avg_loss, accuracy
# Assuming you have your model, dataloader, criterion, and device set up
# Assuming 230k.json is loaded as label_to_index













# model = load_model(model, optimizer)

val_dataset = LaTeXDataset('LaTex_data/split_10', mapping_path, label_to_index_path, transform)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)  # Set shuffle to False for validation
label_to_index = json.load(open(label_to_index_path, 'r'))
index_to_label = {v: k for k, v in label_to_index.items()}  # Reverse the mapping

val_loss, val_accuracy = validate_model(model, val_dataloader, criterion, device, index_to_label)
print(val_accuracy)
save_model(model, optimizer, epoch + 1, val_loss, 'model.pth')

Actual Formula: = \delta / \beta _ { r e l } ^ { s y n c h } \approx \ell _ { P } \, \Rightarrow \, \delta \approx \beta _ { r e l } ^ { s y n c h } \; \ell _ { P } .
Predicted Formula: { _ { _ { 1 } ^ } } { \alpha } t e \delta } = _ { { s } ^ { _ _ r { { { r s } } } ( \ell / } \, } { s } ^ {
--------------------------------------------------
Actual Formula: = \bigcup _ { \alpha \in P / m Q ^ { \vee } } M _ { \alpha } , \quad M _ { \alpha } : = H ( - m ) _ { 1 } \oplus H ( - m ) _ { 2 } \oplus ( \alpha + m Q ^ { \vee } ) \, .
Predicted Formula: { _ { \alpha = 0 } { } { { 2 } } \alpha ( { \alpha } ^ { \; \ge \alpha \alpha } = { \int _ { ) ) \alpha \alpha } , ( { ) , \alpha \alpha } , 1 ) { { 2 } )
--------------------------------------------------
Actual Formula: Q ^ { \alpha } , \bar { Q } _ { \beta } \} = - i ( \Gamma ^ { a } ) _ { \beta } ^ { \alpha } P _ { a } - i ( \Gamma ^ { a b c d e } ) _ { \beta } ^ { \alpha } Z _ { a b c d e } ,
Predicted Formula: ^ { \alpha } , Q ^ Q } ^ { \d

KeyboardInterrupt: 